Survivor WSSYW Ranking
==
This project is a visualization of the Annual Survivor What Season Should You Watch (WSSYW) Thread, which ranks Survivor season's by their general watchability. The goal of this visualization is to elucidate trends in season favorability over time.

All data sourced from [r/survivor](https://www.reddit.com/r/survivor/)

In [132]:
import praw
import re
import pandas as pd
from praw.models import MoreComments
from datetime import datetime, timezone

def utc_to_local(utc_dt):
    return utc_dt.replace(tzinfo=timezone.utc).astimezone(tz=None)

# Create connection to Reddit via PRAW
reddit = praw.Reddit(client_id = 'wxq7j3Y924b3Dg',
                    client_secret = 'qA3FWSMycSm2Jmgq85lLd9RLzAs',
                    user_agent = 'survivor-sbr',
                    username = 'survivorwssywbot',
                    password = 'testpassword12345')
subreddit = reddit.subreddit('survivor')
wssyw_ids = ['10tpq9', # e.g. S06: The Amazon, hyperlinked
             '26viy4', # e.g. Season 28: Cagayan
             '37c7zu', # e.g. S15: China
             '3xp433', 
             '4kzd4g', 
             '5jwzib', 
             '6ga0ty', # [Season 7: Pearl Islands](link to countdown)
             '8p0ye9', 
             'btu8iu'  # Note: Still posting countdown
            ]
wssyw_submissions = []
for r_id in wssyw_ids:
    submission = reddit.submission(id=r_id)
    submission.comments.replace_more(limit=40, threshold=0)
    wssyw_submissions.append(submission)


In [160]:
from datetime import datetime, timezone

def utc_to_local(utc_dt):
    return utc_dt.replace(tzinfo=timezone.utc).astimezone(tz=None)

df = pd.DataFrame(columns = ['Season','1.0','2.0','3.0','4.0','5.0','6.0','7.0','8.0','9.0'])
seasons = [{
    1:'Borneo',
    2:'Australian Outback',
    3:'Africa',
    4:'Marquesas',
    5:'Thailand',
    6:'The Amazon',
    7:'Pearl Islands',
    8:'All-Stars',
    9:'Vanuatu',
    10:'Palau',
    11:'Guatemala',
    12:'Panama',
    13:'Cook Islands',
    14:'Fiji',
    15:'China',
    16:'Micronesia',
    17:'Gabon',
    18:'Tocantins',
    19:'Samoa',
    20:'Heroes vs. Villains',
    21:'Nicaragua',
    22:'Redemption Island',
    23:'South Pacific',
    24:'One World',
    25:'Philippines',
    26:'Caramoan',
    27:'Blood vs. Water',
    28:'Cagayan',
    29:'San Juan del Sur',
    30:'World Apart',
    31:'Cambodia',
    32:'Kaôh Rōng',
    33:'Millennials vs. Gen X',
    34:'Game Changers',
    35:'Heroes vs. Healers vs. Hustlers',
    36:'Ghost Island',
    37:'David vs. Goliath',
    38:'Edge of Extinction'}
]

for post in wssyw_submissions:
    score_dict = {}
    print(utc_to_local(datetime.fromtimestamp(post.created_utc)))
    # For each top level comment, identify season and map karma score
    for top_level_comment in post.comments:
        season = None
        match1 = re.search("(\d+:( (\w|-)+|\.+){1,})",top_level_comment.body)
        match2 = re.search("(:(\d+)( (\w|-)+|\.+){1,})",top_level_comment.body) # Data error of S:00 Example Name
        if match1:
            season = int(match1.group().split(': ')[0])
        elif match2:
            season = int(match2.group().split(':')[1].split(' ')[0])
        if match1 or match2:
            score = top_level_comment.score
            score_dict[season] = score
    seasons.append(score_dict)
    
df = pd.DataFrame(seasons).transpose()
df.columns = ['Season','1.0','2.0','3.0','4.0','5.0','6.0','7.0','8.0','9.0']
df

2012-10-02 09:13:45-04:00
2014-05-30 06:11:20-04:00
2015-05-26 07:42:59-04:00
2015-12-21 02:40:33-05:00
2016-05-25 04:23:09-04:00
2016-12-23 03:02:22-05:00
2017-06-09 09:48:41-04:00
2018-06-06 06:16:21-04:00
2019-05-28 11:00:02-04:00


,Season,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0
1,Borneo,27,28,68,152,158,193,135,142,188
2,Australian Outback,21,18,70,78,80,88,64,83,108
3,Africa,3,11,56,81,66,97,66,77,131
4,Marquesas,5,22,49,70,61,82,54,88,136
5,Thailand,-7,4,-14,-6,5,11,12,12,-10
6,The Amazon,33,51,-14,101,134,144,108,144,166
7,Pearl Islands,58,67,182,287,264,1,258,252,306
8,All-Stars,24,12,6,8,22,1,8,5,14
9,Vanuatu,7,30,58,105,107,113,75,127,158
10,Palau,26,20,79,128,149,141,94,130,151
